# Decision Trees

In [1]:
import numpy as np
np.random.seed(1)

## Classification with DTs

First we'll load the famous *iris* dataset, dealing with plant classification:

In [2]:
from sklearn.datasets import load_iris
iris = load_iris()

Let's look inside of it to see what datatypes scikit-learn wants, and how their sample dataset is formatted, so that we can prepare our own datasets later:

In [3]:
iris.keys()

dict_keys(['data', 'DESCR', 'target', 'feature_names', 'target_names'])

So the data is in dictionary format, and we can access the data and labels by indexing certain keys:

In [4]:
iris.DESCR

'Iris Plants Database\n====================\n\nNotes\n-----\nData Set Characteristics:\n    :Number of Instances: 150 (50 in each of three classes)\n    :Number of Attributes: 4 numeric, predictive attributes and the class\n    :Attribute Information:\n        - sepal length in cm\n        - sepal width in cm\n        - petal length in cm\n        - petal width in cm\n        - class:\n                - Iris-Setosa\n                - Iris-Versicolour\n                - Iris-Virginica\n    :Summary Statistics:\n\n    ============== ==== ==== ======= ===== ====================\n                    Min  Max   Mean    SD   Class Correlation\n    ============== ==== ==== ======= ===== ====================\n    sepal length:   4.3  7.9   5.84   0.83    0.7826\n    sepal width:    2.0  4.4   3.05   0.43   -0.4194\n    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)\n    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)\n    ============== ==== ==== ======= ===== ============

So what are the features, and what are we predicting?

In [5]:
print(iris.feature_names)
print(len(iris.feature_names))
print()
print(iris.target_names)
print(len(iris.target_names))

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
4

['setosa' 'versicolor' 'virginica']
3


So we are using 4 features for each observation, trying to classfiy each observation into one of three categories, using only those 4 features. How are these input features formatted?

In [6]:
print(len(iris.data))
print(type(iris.data))
iris.data

150
<class 'numpy.ndarray'>


array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2],
       [ 5.4,  3.9,  1.7,  0.4],
       [ 4.6,  3.4,  1.4,  0.3],
       [ 5. ,  3.4,  1.5,  0.2],
       [ 4.4,  2.9,  1.4,  0.2],
       [ 4.9,  3.1,  1.5,  0.1],
       [ 5.4,  3.7,  1.5,  0.2],
       [ 4.8,  3.4,  1.6,  0.2],
       [ 4.8,  3. ,  1.4,  0.1],
       [ 4.3,  3. ,  1.1,  0.1],
       [ 5.8,  4. ,  1.2,  0.2],
       [ 5.7,  4.4,  1.5,  0.4],
       [ 5.4,  3.9,  1.3,  0.4],
       [ 5.1,  3.5,  1.4,  0.3],
       [ 5.7,  3.8,  1.7,  0.3],
       [ 5.1,  3.8,  1.5,  0.3],
       [ 5.4,  3.4,  1.7,  0.2],
       [ 5.1,  3.7,  1.5,  0.4],
       [ 4.6,  3.6,  1. ,  0.2],
       [ 5.1,  3.3,  1.7,  0.5],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 5. ,  3. ,  1.6,  0.2],
       [ 5. ,  3.4,  1.6,  0.4],
       [ 5.2,  3.5,  1.5,  0.2],
       [ 5.2,  3.4,  1.4,  0.2],
       [ 4.7,  3.2,  1.6,  0.2],
       [ 4

We have a large numpy array of length 150, one for each observation, and each observation has its own numpy array of length 4, one for each feature. Each inner array *must* lineup with the order of the variables *and* all other arrays. **ORDER MATTERS**.

What about the prediction?

In [7]:
print(len(iris.target))
print(type(iris.target))
iris.target

150
<class 'numpy.ndarray'>


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Again, we have 150 observations, but *no* sub arrays. The target data is one dimension. Order matters here as well, they should correspond to the feature indices in the data array.

Now we split the data into training and testing:

In [8]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target,
                                                    train_size=0.75, test_size=0.25)

/Users/chench/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


After the train/test split, scikit-learn makes the rest of the process easy. We just have to decide on our parameters:

In [9]:
from sklearn import tree

dt_classifier = tree.DecisionTreeClassifier(criterion='gini',  # or 'entropy' for information gain
                       splitter='best',  # or 'random' for random best split
                       max_depth=None,  # how deep tree nodes can go
                       min_samples_split=2,  # samples needed to split node
                       min_samples_leaf=1,  # samples needed for a leaf
                       min_weight_fraction_leaf=0.0,  # weight of samples needed for a node
                       max_features=None,  # number of features to look for when splitting
                       max_leaf_nodes=None,  # max nodes
                       min_impurity_split=1e-07)  # early stopping

model = dt_classifier.fit(X_train, y_train)
print(model.score(X_test, y_test))

0.973684210526


In [10]:
print(model.decision_path(X_test)) # Return a node indicator matrix where non zero elements indicates that the samples goes through the nodes.

  (0, 0)	1
  (0, 1)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (2, 0)	1
  (2, 2)	1
  (2, 3)	1
  (2, 4)	1
  (3, 0)	1
  (3, 1)	1
  (4, 0)	1
  (4, 2)	1
  (4, 10)	1
  (4, 14)	1
  (5, 0)	1
  (5, 2)	1
  (5, 3)	1
  (5, 4)	1
  (6, 0)	1
  (6, 2)	1
  (6, 10)	1
  (6, 14)	1
  (7, 0)	1
  :	:
  (30, 4)	1
  (31, 0)	1
  (31, 2)	1
  (31, 10)	1
  (31, 14)	1
  (32, 0)	1
  (32, 2)	1
  (32, 3)	1
  (32, 4)	1
  (33, 0)	1
  (33, 2)	1
  (33, 10)	1
  (33, 14)	1
  (34, 0)	1
  (34, 2)	1
  (34, 10)	1
  (34, 14)	1
  (35, 0)	1
  (35, 1)	1
  (36, 0)	1
  (36, 2)	1
  (36, 3)	1
  (36, 4)	1
  (37, 0)	1
  (37, 1)	1


### Grid Search

In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = {'min_samples_split': range(2,10),
              'min_samples_leaf': range(1,10)}

model_c = GridSearchCV(tree.DecisionTreeClassifier(), param_grid)
model_c.fit(X_train, y_train)

best_index = np.argmax(model_c.cv_results_["mean_test_score"])

print(model_c.cv_results_["params"][best_index])
print(max(model_c.cv_results_["mean_test_score"]))
print(model_c.score(X_test, y_test))

{'min_samples_leaf': 1, 'min_samples_split': 3}
0.9375
0.973684210526


## Regression with DTs

### Dataset and prep

For demonstration, we will use a boston housing dataset, which comes with scikit-learn:

In [12]:
from sklearn.datasets import load_boston

boston = load_boston()

If you are going to follow along in other tutorials in the scikit-learn documentation, you will need to know the data structures used as inputs to the models. Let'see what's in the boston dataset:

In [13]:
boston.keys()

dict_keys(['data', 'DESCR', 'target', 'feature_names'])

The description will tell us more about the dataset:

In [14]:
boston.DESCR

"Boston House Prices dataset\n===========================\n\nNotes\n------\nData Set Characteristics:  \n\n    :Number of Instances: 506 \n\n    :Number of Attributes: 13 numeric/categorical predictive\n    \n    :Median Value (attribute 14) is usually the target\n\n    :Attribute Information (in order):\n        - CRIM     per capita crime rate by town\n        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.\n        - INDUS    proportion of non-retail business acres per town\n        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)\n        - NOX      nitric oxides concentration (parts per 10 million)\n        - RM       average number of rooms per dwelling\n        - AGE      proportion of owner-occupied units built prior to 1940\n        - DIS      weighted distances to five Boston employment centres\n        - RAD      index of accessibility to radial highways\n        - TAX      full-value property-tax rate per $10,000\n

So we are working on predicitng median value of a home from 506 observations, and 13 covariates including crime rate, lot size, industry/commercial proportion, presence of the Charles River, nitric oxide concentration, rooms per dwelling, units built before 1940, distance to employment centers, access to highways, tax rate, school proxy, black population, and status. To get the variable names we can ask for them in the dictionary:

In [15]:
print(boston.feature_names)
print()
print(type(boston.feature_names))
print()
print(len(boston.feature_names))

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']

<class 'numpy.ndarray'>

13


We see the input is a numpy array of strings for the variable labels. To get the variable data, we ask the dictionary for the data:

In [16]:
print(boston.data)
print()
print(type(boston.data))
print()
print(len(boston.data))

[[  6.32000000e-03   1.80000000e+01   2.31000000e+00 ...,   1.53000000e+01
    3.96900000e+02   4.98000000e+00]
 [  2.73100000e-02   0.00000000e+00   7.07000000e+00 ...,   1.78000000e+01
    3.96900000e+02   9.14000000e+00]
 [  2.72900000e-02   0.00000000e+00   7.07000000e+00 ...,   1.78000000e+01
    3.92830000e+02   4.03000000e+00]
 ..., 
 [  6.07600000e-02   0.00000000e+00   1.19300000e+01 ...,   2.10000000e+01
    3.96900000e+02   5.64000000e+00]
 [  1.09590000e-01   0.00000000e+00   1.19300000e+01 ...,   2.10000000e+01
    3.93450000e+02   6.48000000e+00]
 [  4.74100000e-02   0.00000000e+00   1.19300000e+01 ...,   2.10000000e+01
    3.96900000e+02   7.88000000e+00]]

<class 'numpy.ndarray'>

506


The data is a numpy array, inside of which there is a separate array for each observation (all 506 for each hous, *not* 13 for each variable). Each inner array *must* lineup with the order of the variables *and* all other arrays. **ORDER MATTERS**

The target, or *y* is accessed in the dictionary as well:

In [17]:
print(boston.target)
print()
print(type(boston.target))
print()
print(len(boston.target))

[ 24.   21.6  34.7  33.4  36.2  28.7  22.9  27.1  16.5  18.9  15.   18.9
  21.7  20.4  18.2  19.9  23.1  17.5  20.2  18.2  13.6  19.6  15.2  14.5
  15.6  13.9  16.6  14.8  18.4  21.   12.7  14.5  13.2  13.1  13.5  18.9
  20.   21.   24.7  30.8  34.9  26.6  25.3  24.7  21.2  19.3  20.   16.6
  14.4  19.4  19.7  20.5  25.   23.4  18.9  35.4  24.7  31.6  23.3  19.6
  18.7  16.   22.2  25.   33.   23.5  19.4  22.   17.4  20.9  24.2  21.7
  22.8  23.4  24.1  21.4  20.   20.8  21.2  20.3  28.   23.9  24.8  22.9
  23.9  26.6  22.5  22.2  23.6  28.7  22.6  22.   22.9  25.   20.6  28.4
  21.4  38.7  43.8  33.2  27.5  26.5  18.6  19.3  20.1  19.5  19.5  20.4
  19.8  19.4  21.7  22.8  18.8  18.7  18.5  18.3  21.2  19.2  20.4  19.3
  22.   20.3  20.5  17.3  18.8  21.4  15.7  16.2  18.   14.3  19.2  19.6
  23.   18.4  15.6  18.1  17.4  17.1  13.3  17.8  14.   14.4  13.4  15.6
  11.8  13.8  15.6  14.6  17.8  15.4  21.5  19.6  15.3  19.4  17.   15.6
  13.1  41.3  24.3  23.3  27.   50.   50.   50.   2

The target array is only one dimmension, lined up in order with the with the observations in the data array.

Now that we're familiar with the input data, we need to split it up for training and testing:

In [18]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target,
                                                    train_size=0.75, test_size=0.25)

Now we have 75% of the data as training data, and 25% of the data as testing data:

In [19]:
print(len(X_train), len(y_train))
print()
print(len(X_test), len(y_test))

379 379

127 127


In scikit-learn, as soon as you have `X_train`, `X_test`, `y_train`, and `y_test`, everything else is just a matter of choosing parameters for whichever model you choose. But this should not be trivialized, selecting models and that model's parameters is *very* important. While we will not cover it here, you should always select the model and parameters best suited for your data.

### Decision Tree Regression

In [20]:
from sklearn import tree

dt_reg = tree.DecisionTreeRegressor(criterion='mse',  # how to measure fit
                                    splitter='best',  # or 'random' for random best split
                                    max_depth=None,  # how deep tree nodes can go
                                    min_samples_split=2,  # samples needed to split node
                                    min_samples_leaf=1,  # samples needed for a leaf
                                    min_weight_fraction_leaf=0.0,  # weight of samples needed for a node
                                    max_features=None,  # number of features to look for when splitting
                                    max_leaf_nodes=None,  # max nodes
                                    min_impurity_split=1e-07)  # early stopping

model = dt_reg.fit(X_train, y_train)
print(model.score(X_test, y_test))

0.857589194232


In [21]:
print(model.decision_path(X_train))

  (0, 0)	1
  (0, 634)	1
  (0, 635)	1
  (0, 636)	1
  (0, 637)	1
  (0, 659)	1
  (0, 660)	1
  (0, 661)	1
  (0, 662)	1
  (0, 663)	1
  (0, 664)	1
  (0, 665)	1
  (1, 0)	1
  (1, 1)	1
  (1, 2)	1
  (1, 40)	1
  (1, 42)	1
  (1, 332)	1
  (1, 340)	1
  (1, 341)	1
  (1, 351)	1
  (1, 352)	1
  (1, 353)	1
  (1, 354)	1
  (1, 355)	1
  :	:
  (377, 1)	1
  (377, 387)	1
  (377, 487)	1
  (377, 488)	1
  (377, 489)	1
  (377, 491)	1
  (377, 505)	1
  (377, 525)	1
  (377, 526)	1
  (377, 542)	1
  (377, 543)	1
  (377, 544)	1
  (378, 0)	1
  (378, 1)	1
  (378, 2)	1
  (378, 40)	1
  (378, 42)	1
  (378, 43)	1
  (378, 44)	1
  (378, 46)	1
  (378, 47)	1
  (378, 48)	1
  (378, 49)	1
  (378, 53)	1
  (378, 54)	1


### Grid Search

In [22]:
param_grid = {'min_samples_split': range(2,10),
              'min_samples_leaf': range(1,10)}

model_r = GridSearchCV(tree.DecisionTreeRegressor(), param_grid)
model_r.fit(X_train, y_train)

best_index = np.argmax(model_r.cv_results_["mean_test_score"])

print(model_r.cv_results_["params"][best_index])
print(max(model_r.cv_results_["mean_test_score"]))
print(model_r.score(X_test, y_test))

{'min_samples_leaf': 5, 'min_samples_split': 4}
0.751487537371
0.812122447534
